In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install tensorflow-gpu==1.14.0

import tensorflow as tf
tf.VERSION

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

'1.14.0'

# Abstract
This is a convolutional neural network that measures population density from birds-eye view images. It takes-in an input image and then detects the population density into 4 outputs:
- 0 people = Zero Density
- 1 to 5 people = Low Density
- 6 to 30 = Medium Density
- 31 to infinity = High Density

# Defining Constants
Let's define some constants to use in our neural network for later.

In [0]:
# import numpy
import numpy as np

# set the random number generator of numpy
from numpy.random import seed
seed(1)

In [0]:
# how aggressive will be the data augmentation / transformation
transformation_ratio = .05

In [0]:
# number of training and testing images
num_training_samples = 95
num_testing_samples = 60

# number of images to process before the weights are updated: try 4, 8, 16, 32, etc. depending on the CPU/GPU memory capacity
training_batch_size = 16
testing_batch_size = 8

# image's square dimension for the neural network (width x height)
image_size = 224

# steps is the number of images per epoch
training_steps = np.ceil(num_training_samples / training_batch_size)
testing_steps = np.ceil(num_testing_samples / testing_batch_size)

# Pre-Processing Image Data
Let's pre-process image data using Image Augmentation from Keras's ImageDataGenerator class.

Image augmentation allows us to create many batches of the images, which create many more diverse set of the images. Some augmentations could be rotating, stretching, zooming, etc.

This helps prevent overfitting because augmentation better diversifies the data set.

In [0]:
# import keras
import tensorflow.keras as keras

# seed the random number generator of tensorflow
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# import the keras image data augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# create a data generator for the training set
training_batches = ImageDataGenerator(
    rescale= 1.0 / 255,
    rotation_range = transformation_ratio,
    shear_range = transformation_ratio,
    zoom_range = transformation_ratio,
    cval = transformation_ratio,
    horizontal_flip=True,
    vertical_flip=True,
).flow_from_directory(
    "drive/My Drive/notebooks/images/training",
    target_size = (image_size, image_size),
    batch_size = training_batch_size
)

Found 95 images belonging to 4 classes.


In [9]:
# create a data generator for the testing set using the built-in mobilenet functions
testing_batches = ImageDataGenerator(
    rescale = 1.0 / 255
).flow_from_directory(
    "drive/My Drive/notebooks/images/testing",
    target_size = (image_size, image_size),
    batch_size = testing_batch_size
)

Found 60 images belonging to 4 classes.


# Convolutional Neural Network
Let's create a convolutional neural network to classify the image based on the categories.

The below architecture is a VGG16 neural network.

In [0]:
# import the basic model class
from tensorflow.keras.models import Model

# import the xception model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# import layers
from tensorflow.keras.layers import GlobalMaxPooling2D, Dropout, Dense

In [11]:
# use the pre-tained MobileNetV2 model
base_model = MobileNetV2(input_shape = (image_size, image_size, 3), weights = "imagenet", include_top = False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
9412608/9406464 [==============================] - 1s 0us/step


In [0]:
# add a MaxPooling layer
pool_layer = GlobalMaxPooling2D()(base_model.output)

# add a Dropout layer to prevent overfitting
dropout_layer = Dropout(rate=0.20)(pool_layer)

# add the output layer to the neural network
output_layer = Dense(units = 4, activation = "softmax")(dropout_layer)
model = Model(base_model.input, output_layer)

In [0]:
# do not train the layers from the base (original) Xception model
for layer in base_model.layers:
    layer.trainable = False

In [0]:
# compile the model using the nadam (stochastic Gradient Descent) optimizer
model.compile(
    optimizer="nadam",
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

# Callbacks
Let's define Callbacks to save the neural network whenever we reached the most accurate version of the model.

In [0]:
# import the callbacks for the models to use to fit to the training set
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [0]:
# declare a checkpoint to save the best version of the model
model_file = "model.h5"
checkpoint = ModelCheckpoint(model_file, monitor = "val_acc", save_best_only = True, verbose = 1, mode = "max")

# early stop the model as the validation accuracy stagnates
# early_stop = EarlyStopping(monitor = "val_acc", patience = 5, verbose = 0)

# reduce the learning rate as the learning stagnates
# reduce_lr = ReduceLROnPlateau(monitor="val_acc", factor=0.5, patience=2, mode="max", min_lr=0.00001)

callbacks_list = [checkpoint]

# Fitting The Model
Let's fit the model to the training set.

In [0]:
classifier = model.fit_generator(
    training_batches,
    steps_per_epoch = training_steps,
    validation_data = testing_batches,
    validation_steps = testing_steps,
    epochs = 100,
    verbose = 1,
    callbacks = callbacks_list
)

Epoch 1/100
5/6 [========================>.....] - ETA: 5s - loss: 5.3868 - acc: 0.2785 
Epoch 00001: val_acc improved from -inf to 0.30000, saving model to model.h5
6/6 [==============================] - 57s 9s/step - loss: 5.3986 - acc: 0.2947 - val_loss: 2.7298 - val_acc: 0.3000
Epoch 2/100
5/6 [========================>.....] - ETA: 0s - loss: 3.9080 - acc: 0.3544
Epoch 00002: val_acc improved from 0.30000 to 0.35000, saving model to model.h5
6/6 [==============================] - 2s 388ms/step - loss: 3.9222 - acc: 0.3474 - val_loss: 3.2159 - val_acc: 0.3500
Epoch 3/100
5/6 [========================>.....] - ETA: 0s - loss: 3.1741 - acc: 0.3544
Epoch 00003: val_acc did not improve from 0.35000
6/6 [==============================] - 3s 505ms/step - loss: 3.1770 - acc: 0.3895 - val_loss: 2.6798 - val_acc: 0.3500
Epoch 4/100
5/6 [========================>.....] - ETA: 0s - loss: 3.6373 - acc: 0.4430
Epoch 00004: val_acc improved from 0.35000 to 0.46667, saving model to model.h5
6/6 [